In [1]:
import geopandas as gpd

wards = gpd.read_file("data/cpt_boundary_wards.gpkg")
parks = gpd.read_file("data/cpt_parks.gpkg")

In [3]:
# Step 2: Ensure geometries are valid
wards = wards[wards.is_valid]
parks = parks[parks.is_valid]

# Step 3: Reproject to Web Mercator (EPSG:3857)
wards = wards.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [4]:
# Step 4: Spatial intersection – get parks within wards
parks_in_wards = gpd.overlay(parks, wards, how="intersection")

In [5]:
wards.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   OBJECTID   116 non-null    int32   
 1   WARD_NAME  116 non-null    object  
 2   WARD_YEAR  116 non-null    int32   
 3   geometry   116 non-null    geometry
dtypes: geometry(1), int32(2), object(1)
memory usage: 2.8+ KB


In [6]:
# Step 5: Calculate green area per ward
parks_in_wards["green_area"] = parks_in_wards.geometry.area

green_area_by_ward = parks_in_wards.groupby("WARD_NAME")["green_area"].sum().reset_index()

In [7]:
# Step 6: Calculate total ward area
wards["total_area"] = wards.geometry.area

# Merge green area values into wards GeoDataFrame
wards = wards.merge(green_area_by_ward, on="WARD_NAME", how="left")

# Fill NaNs in green_area with 0 (wards with no green space)
wards["green_area"] = wards["green_area"].fillna(0)

# Calculate green ratio
wards["green_ratio"] = wards["green_area"] / wards["total_area"]

In [8]:
# Step 7: Save to GeoPackage
wards.to_file("wards_with_green_ratio.gpkg", layer="wards_green_ratio", driver="GPKG")